# Partie 3 - Modele Predictif 2 : Logistic Regression

## Presentation du Modele

La régression logistique est un algorithme utilisé pour prédire la probabilité qu'une donnée appartienne à une classe (comme "oui" ou "non"). Elle utilise une fonction appelée sigmoïde pour transformer une combinaison des caractéristiques d'entrée en une valeur entre 0 et 1, représentant cette probabilité. L'algorithme ajuste ses paramètres pour minimiser les erreurs de prédiction sur les données d'entraînement. 

# LG : Logistic Regression

On commence par importer les librairies nécessaires, donc numpy et pandas. Ensuite, on importe le dataset.

In [27]:
import pandas as pd
import numpy as np

data = pd.read_csv('heart_disease_by_ceo.csv', index_col=0)

## Selection des "features" et de la valeur "cible"

On va ensuite selectionner la valeur cible (ici la colonne "HeartDisease") et les features (toutes les colonnes sauf "HeartDisease").


In [28]:
X = data.drop(columns=["HeartDisease"])
Y = data["HeartDisease"]

## Separation des données en données d'entrainement et de test

On va ensuite entrainer le modele Logistic Regression sur les données d'entrainement à l'aide de train_test_split.

On va proceder de la maniere suivante, 80% de données pour l'entrainement et 20% pour le test. Comme pour le modèle KNN.

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

## Normalisation des données

On va ensuite Normaliser les données d'entrainement et de test à l'aide de StandardScaler afin de les mettre à la même échelle. Cette étape est très importante pour les modèles de Regression Logistique.

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## Entrainement du modèle

On va ensuite entrainer le modèle Logistic Regression sur les données d'entrainement à l'aide de la fonction fit(). C'est a partir de cette étape que le modèle change par rapport au modèle KNN.

In [31]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, Y_train)


LogisticRegression()

## Prédictions

On va ensuite faire des prédictions sur les données de test à l'aide de la fonction predict().

In [32]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(X_test)

## Evaluation du modèle

On va enfin évaluer le modèle en utilisant la fonction accuracy_score() pour calculer la précision du modèle et classification_report() pour afficher les résultats.


In [33]:
print(accuracy_score(Y_test, y_pred))
print("\n")
print("---------------------------------------")
print("\n")
print(classification_report(Y_test, y_pred))

0.8695652173913043


---------------------------------------


              precision    recall  f1-score   support

           0       0.80      0.89      0.84        72
           1       0.92      0.86      0.89       112

    accuracy                           0.87       184
   macro avg       0.86      0.87      0.87       184
weighted avg       0.87      0.87      0.87       184



Puis la matrice de confusion pour voir les résultats de manière plus visuelle.

In [34]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, y_pred))
print("\n")
print("---------------------------------------")
print("\n")
print(pd.crosstab(Y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True))

[[64  8]
 [16 96]]


---------------------------------------


Predicted   0    1  All
Actual                 
0          64    8   72
1          16   96  112
All        80  104  184


## Analyse des coefficients du modèle

Avant de tester notre modèle sur des cas spécifiques, analysons les coefficients pour comprendre comment chaque caractéristique influence la prédiction. Ces coefficients nous montrent l'importance relative et la direction (positive ou négative) de l'impact de chaque variable sur le risque de maladie cardiaque.

In [35]:
# Récupérer les noms des caractéristiques
feature_names = X.columns.tolist()

# Récupérer les coefficients du modèle
coefficients = model.coef_[0]

# Créer un DataFrame pour associer chaque caractéristique à son coefficient
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Trier par valeur absolue du coefficient (importance)
coef_df['Abs_Coefficient'] = abs(coef_df['Coefficient'])
coef_df = coef_df.sort_values('Abs_Coefficient', ascending=False)

print("Les caractéristiques les plus importantes selon leurs coefficients:")
print(coef_df)

Les caractéristiques les plus importantes selon leurs coefficients:
              Feature  Coefficient  Abs_Coefficient
1                 Sex    -0.631972         0.631972
15        Up_ST_Slope    -0.610622         0.610622
16      Flat_ST_Slope     0.587651         0.587651
11      ASY_ChestPain     0.502880         0.502880
4           FastingBS     0.500162         0.500162
6      ExerciseAngina     0.437934         0.437934
7             Oldpeak     0.407109         0.407109
9       ATA_ChestPain    -0.313199         0.313199
10      NAP_ChestPain    -0.248847         0.248847
5               MaxHR    -0.171679         0.171679
0                 Age     0.141115         0.141115
8        TA_ChestPain    -0.111142         0.111142
3         Cholesterol    -0.047843         0.047843
17      Down_ST_Slope     0.034008         0.034008
13      ST_RestingECG     0.026788         0.026788
14     LVH_RestingECG    -0.012628         0.012628
12  Normal_RestingECG    -0.010936         0.010

### Interprétation des coefficients


Les coefficients positifs indiquent des facteurs qui augmentent le risque de maladie cardiaque, tandis que les coefficients négatifs indiquent des facteurs protecteurs.

Principaux facteurs de risque (coefficients positifs les plus élevés):
- ASY_ChestPain: Les douleurs thoraciques asymptomatiques sont un indicateur significatif de maladie cardiaque
- FastingBS: Une glycémie à jeun élevée augmente considérablement le risque
- Flat_ST_Slope: Une pente ST plate à l'ECG est associée à un risque accru

Principaux facteurs protecteurs (coefficients négatifs les plus importants):
- Sex: Les femmes présentent un risque plus faible que les hommes
- Up_ST_Slope: Une pente ST ascendante est associée à un risque réduit
- MaxHR: Une fréquence cardiaque maximale élevée indique une meilleure condition cardiovasculaire

Cette analyse des coefficients illustre la grande valeur de la régression logistique: sa capacité à fournir des informations interprétables sur les facteurs de risque.

## Test du modèle

On va maintenant tester le modèle sur un exemple de données pour voir si le modèle prédit correctement la valeur cible.

In [36]:
def tester_patient(liste_colonne, patient):
    df = pd.DataFrame(patient, columns=liste_colonne)
    df = scaler.transform(df)
    prediction = model.predict(df)
    print(f"Prédiction : {'Malade' if prediction[0] == 1 else 'Non Malade'}")

def tester_plusieurs_patients(liste_colonne, patients):
    index = 0
    for patient in patients:
        print(f"Patient {index}")
        tester_patient(liste_colonne, patient)
        print("\n")
        index += 1
    

In [37]:
liste_colonnes = ['Age', 'Sex', 'RestingBP', 'Cholesterol', 'FastingBS', 
                  'MaxHR', 'ExerciseAngina', 'Oldpeak', 'TA_ChestPain', 
                  'ATA_ChestPain', 'NAP_ChestPain', 'ASY_ChestPain', 
                  'Normal_RestingECG', 'ST_RestingECG', 'LVH_RestingECG', 
                  'Up_ST_Slope', 'Flat_ST_Slope', 'Down_ST_Slope']


patient = [[40,0,140,289,0,172,0,0.0,0,1,0,0,1,0,0,1,0,0]] #0 Non malade
patient2 = [[57,1,130,236,0,174,0,0.0,0,1,0,0,0,0,1,0,1,0]] #916 malade
patient3 = [[57,0,130,131,0,115,1,1.2,0,0,0,1,1,0,0,0,1,0]] #915 malade
patient4 = [[59,0,164,176,1,90,0,1.0,0,0,0,1,0,0,1,0,1,0]] #911 malade
patient5 = [[49,1,160,180,0,156,0,1.0,0,0,1,0,1,0,0,0,1,0]] #1 malade

patients = [patient, patient2, patient3, patient4, patient5]
tester_plusieurs_patients(liste_colonnes, patients)

Patient 0
Prédiction : Non Malade


Patient 1
Prédiction : Non Malade


Patient 2
Prédiction : Malade


Patient 3
Prédiction : Malade


Patient 4
Prédiction : Non Malade




### Optimisation du modele

83% d'accuracy, c'est pas mal mais on peut faire mieux. On va essayer d'optimiser le modèle en utilisant GridSearchCV pour trouver les meilleurs hyperparamètres.

Explication de la valeur C : 
C: float, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.


On va ensuite entrainer le modèle Logistic Regression sur les données d'entrainement à l'aide de GridSearchCV.

Dans GridSearchCV, on va utiliser la grille de paramètres suivants :

- 'C': [0.1, 1, 10, 100, 1000] qui correspond à l'inverse de la force de régularisation. Plus la valeur de C est petite, plus le modèle est régularisé.

- 'penalty': ['l1', 'l2'] qui correspond à la norme de la régularisation. L1 est la régularisation Lasso et L2 est la régularisation Ridge.

- 'solver': ['liblinear', 'saga'] qui correspond à l'optimiseur utilisé pour résoudre le problème d'optimisation. liblinear est utilisé pour les petites datasets et saga pour les grands datasets.







In [38]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],  
    'solver': ['liblinear','saga']  
}
model = LogisticRegression(max_iter=1000)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, Y_train)
results = pd.DataFrame(grid_search.cv_results_)

print("Best Parameters:", grid_search.best_params_)


best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print(f"Best Model Accuracy: {accuracy:.4f}")
print(classification_report(Y_test, y_pred))


Best Parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}
Best Model Accuracy: 0.8641
              precision    recall  f1-score   support

           0       0.79      0.89      0.84        72
           1       0.92      0.85      0.88       112

    accuracy                           0.86       184
   macro avg       0.86      0.87      0.86       184
weighted avg       0.87      0.86      0.87       184



# Conclusion et Note

## Note

On va maintenant noter notre modele à l'aide des criteres de la partie 2 :

- Performance : 
    - Precision : 80 % pour les nons malades et 92 % pour les malades.
    - Accuracy : 87 %
    - Recall : 89% pour les non malades et 86 % pour les malades.
    - F1 Score : 84 % pour les non malades et 89 % pour les malades.
    - Capacite a reconnaitre les faux positifs et les faux negatifs : Sur les 5 patients tests, 4 etaient malades et 1 non malade. Le non malade a ete reconnu comme non malade. Mais parmis les malades, 2 ont ete reconnus comme malades et 2 comme non malades.

Les resultats sont assez bons, mais un peu en dessous de ceux de KNN. je donnerais donc une note de 1/2.

Robustesse :  Le modele est assez robuste, mais il est un peu plus sensible aux outliers que KNN. Je donnerais donc une note de 0/1.

Complexité et Rapidité : Le modele est assez complexe mais contrairement a KNN, il est rapide à entrainer et tester peut importe la taille des données. Je donnerais donc une note de 1/1.

Interpretabilité et Explicabilité :  Dans le cas de la regression logistique, on peut facilement interpreter les resultats à l'aide des coefficients. Je donnerais donc une note de 1/1.

La note finale est donc de 3/5.

## Avantages

La logistique regression a beaucoup d'avantages, notamment le fait qu'il soit rapide à entrainer et à predire mais egalement qu'il soit facile à interpreter et à expliquer. De plus il est fonctionne assez bien peu importe la taille du jeu de données.


## Inconvénients

Cependant, dans notre cas precisement il est beaucoup plus limité. En effet, dans notre dataset il y 'a beaucoup de relation non linéaire donc la performance du modèle est limitée. De plus il est assez sensible aux valeurs aberrantes ce qui peut fausser les resultats. Enfin, il est très mauvais dans le cas des variables categoriques complexes.